# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys

In [3]:
import sagemaker
import pandas as pd

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)

train_val_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_train_val.csv')).drop_duplicates()
#tot_df = tot_df[tot_df]

test_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_test_v0.7.1.csv'))[['excerpt']]

/home/selim/anaconda3/envs/deepl/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
"""from ast import literal_eval

train_val_df['target'] = train_val_df['target'].apply(
    lambda x: [item for item in literal_eval(x) if 'first_level' in item]
)
train_val_df.to_csv(os.path.join(DATA_PATH, 'tmp_train_val.csv'))

"""
#train_val_df = pd.read_csv(os.path.join(DATA_PATH, 'tmp_train_val.csv'))
#train_val_df['target']

"from ast import literal_eval\n\ntrain_val_df['target'] = train_val_df['target'].apply(\n    lambda x: [item for item in literal_eval(x) if 'first_level' in item]\n)\ntrain_val_df.to_csv(os.path.join(DATA_PATH, 'tmp_train_val.csv'))\n\n"

In [8]:
from ast import literal_eval
def flatten(t):
    return [item for sublist in t for item in sublist]

len(list(set(flatten(train_val_df['target'].apply(literal_eval)))))

103

In [9]:
columns = ['excerpt', 'entry_id', 'target']
train_val_df = train_val_df[columns]

In [10]:
"""tot_df = pd.concat(
    [
        tot_df,
        pd.read_csv(os.path.join(DATA_PATH, 'augmented_subpillars.csv'))
    ]
)"""

"tot_df = pd.concat(\n    [\n        tot_df,\n        pd.read_csv(os.path.join(DATA_PATH, 'augmented_subpillars.csv'))\n    ]\n)"

In [11]:
train_val_df.columns

Index(['excerpt', 'entry_id', 'target'], dtype='object')

In [12]:
train_val_df.shape

(422160, 3)

## Sagemaker Prep

### Session

Configure SageMaker

In [13]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [14]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [15]:
sample = False  # To make the computations faster, sample = True.

if sample:
    train_val_df = train_val_df.sample(n=20_000)
    
job_name = f"pytorch-{formatted_time()}-all-models"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')

train_val_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
test_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [16]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

# CPU instances
instances = [
    'ml.c4.2xlarge',
    'ml.c4.4xlarge',
    'ml.c5n.2xlarge'
]

# https://aws.amazon.com/sagemaker/pricing/instance-types

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [17]:
from sagemaker.pytorch import PyTorch

instance_type = "ml.p3.2xlarge"

hyperparameters = {
    "tracking_uri": MLFLOW_SERVER,
    "experiment_name": "pl-new-architecture",
    "max_len": 128,
    "epochs": 2,
    "model_name": "nreimers/mMiniLMv2-L6-H384-distilled-from-XLMR-Large",
    "tokenizer_name": "nreimers/mMiniLMv2-L6-H384-distilled-from-XLMR-Large",
    "dropout": 0.2,
    "output_length": 384,
    "learning_rate": 1e-5,
    "weight_decay": 1e-2,
    "instance_type": instance_type,
    "f_beta": 0.7,
    "nb_repetitions": 1,
    "run_name": "all_tags_final",
    "train_batch_size": 32,
    "val_batch_size": 64,
}

estimator = PyTorch(
    entry_point="train_mlflow.py",
    source_dir=str(
        "../../../scripts/training/selim/multiclass-lightning/MultitaskAllInOne"
    ),
    output_path=str(DEV_BUCKET / "models/"),
    code_location=str(input_path),
    instance_type=instance_type,
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py3",
    hyperparameters=hyperparameters,
    job_name=job_name,
    #     train_instance_count=2,
    #     train_instance_type="ml.c4.xlarge",
)


In [18]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [19]:
# Fit the estimator
estimator.fit(fit_arguments, job_name=job_name)

2022-07-07 11:39:20 Starting - Starting the training job...ProfilerReport-1657193958: InProgress
...
2022-07-07 11:40:17 Starting - Preparing the instances for training......
2022-07-07 11:41:21 Downloading - Downloading input data...
2022-07-07 11:41:58 Training - Downloading the training image........................
2022-07-07 11:46:03 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-07-07 11:46:06,328 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-07-07 11:46:06,351 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-07-07 11:46:06,358 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-07-07 11:46:06,808 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r r

Job ended with status 'Stopped' rather than 'Completed'. This could mean the job timed out or stopped early for some other reason: Consider checking whether it completed as you expect.


Training seconds: 710
Billable seconds: 710
